<a href="https://colab.research.google.com/github/XxTeosxX/amigo-do-rh/blob/main/amigo_do_rh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para esse projeto vamos ter que instalar dois pacotes externos ao colab  
1 google-generativeai - para fazemos uso das apis de IA do google  
2 pypdf - para poder ler arquivos pdf em forma de texto

In [41]:
!pip install -q -U google-generativeai
!pip install -q -U pypdf

In [51]:
import requests
import google.generativeai as genai
import numpy as np
import pandas as pd
from pypdf import PdfReader
import textwrap

A chave da api foi inserida no colab como visto no final da aula 4  
https://youtu.be/iwt4bOIHy7s?t=4682

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

Vamos fazer o download do pdf da clt do site da camara.  
Fonte do pdf da clt https://bd.camara.leg.br/bd/handle/bdcamara/37179

In [ ]:
!wget https://bd.camara.leg.br/bd/bitstream/handle/bdcamara/37179/consolidacao_leis_trabalho_4ed.pdf

Vamos ler o pdf e colocar o texto dentro de um dataframe do pandas  

In [15]:
leitor = PdfReader("consolidacao_leis_trabalho_4ed.pdf")
numero_de_paginas = len(leitor.pages)

#comecando da pagina dois pois a página 0 é capa e página 1 é contra capa e a página 2 é outra capa interna
inicio = 3
# finalizando na ultima página com alguma informação pois a ultima pagina é capa
fim = numero_de_paginas -1

paginas = []
for numero_da_pagina in range(inicio, fim):
  paginas.append({
      'pagina': numero_da_pagina,
      'texto': leitor.pages[numero_da_pagina].extract_text()
  })

In [16]:
df = pd.DataFrame(paginas)
df.columns = ["pagina", "texto"]

Feito isso vamos agora gerar os embedding para os textos.

In [22]:
def embed_fn(text, model):
  return genai.embed_content(model=model, content=text, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [23]:
model = "models/embedding-001"

In [24]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["texto"], model), axis=1)

Agora aqui esta sendo feita a função para gerar a consulta e um teste para saber se a consulta funcionando como deveria.

In [108]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  # indice = np.argmax(produtos_escalares)
  indices_dos_3_maiores = np.argpartition(produtos_escalares, -3)[-3:]
  return [df.iloc[indice]["texto"] for indice in indices_dos_3_maiores]

In [34]:
consulta = "Quantas vezes no ano eu posso tirar férias? Eu posso dividir os periodos?"
trecho = gerar_e_buscar_consulta(consulta, df, model)

Configurações do modelo e criação do modelo de conversa.

In [76]:
generation_config = {
  "temperature": 1,
  "candidate_count": 1
}

In [77]:
model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)

Uma função para formatar o tamanho da resposta

In [99]:
def print_com_quebra_de_linha(texto_longo):
  largura_linha = 120
  linhas_quebradas = textwrap.wrap(texto_longo, largura_linha)

  for linha in linhas_quebradas:
    print(linha)
  print()


A função que é o amigo do rh.
Aqui vai fazer uso dos dois modelos (embedding e conversa) e vai ficar aguardando uma entrada de pergunta no chat.

In [111]:
prompt_inicial = """Você é um experiente profissional que trabalha em uma empresa e responde dúvidas dos funcionários e gerencia de uma forma mais simples,
usando como base o seguinte texto da CLT:
'{trechos}'.

Responda à seguinte pergunta de um funcionário: '{consulta}'"""

In [112]:
def amigo_do_rh(embedding_model, chat_model):
  print('Este é o seu amigo do rh ele vai ler a CLT e responder perguntas de forma fácil de entender.')
  print('Faça perguntas do tipo : ')
  print('Posso prorrogar a experiência do meu funcionário mais de duas vezes?')
  print('Qual é o periodo de experiência?')
  print('Obs: Para encerrar a conversa com o amigo do rh. Digite fim e enter\n')

  chat = chat_model.start_chat(history=[])

  while True:
    consulta = input('>>> Esperando pergunda:\n')
    if consulta == 'fim':
      break
    trechos = gerar_e_buscar_consulta(consulta, df, embedding_model)
    prompt = prompt_inicial.format(trechos=trechos, consulta=consulta)
    response = chat.send_message(prompt)
    print_com_quebra_de_linha(response.text)

In [113]:
amigo_do_rh(model, model_2)

Este é o seu amigo do rh ele vai ler a CLT e responder perguntas de forma fácil de entender.
Faça perguntas do tipo : 
Posso prorrogar a experiência do meu funcionário mais de duas vezes?
Qual é o periodo de experiência?
Obs: Para encerrar a conversa com o amigo do rh. Digite fim e enter

>>> Esperando pergunda:
Posso prorrogar a experiência do meu funcionário mais de duas vezes?
Não, o texto da CLT não permite a prorrogação da experiência por mais de duas vezes.

>>> Esperando pergunda:
fim
